<div style="background-color: white; padding: 10px;">
<center>
    <img style="padding-right:15px" height='50px' src="https://kartai.no/wp-content/uploads/2025/03/cropped-KartAi-med-partnere-2048x1145.png">
    <img style="padding-left:15px"  height='50px' src="https://www.norkart.no/hubfs/norkart-logo-default.svg">
    </center>
</div>

# 🦜Deus ex geomachina - Learn how to use language models to get geomatics superpowers 🗺️
<a target="_blank" href="https://colab.research.google.com/github/kartAI/deus-ex-geomachina/blob/main/deus_ex_geomachina.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


How do you avoid hallucinations? How can language models perform GIS analyses?

Join a practical workshop where you learn to combine the power of modern AI with geographic data and analyses. During this session, you will:

* Learn how large language models (LLMs) can transform and streamline geographic analyses
* Get hands-on experience connecting ChatGPT-like models to PostGIS databases
* Explore how to ask complex geographic questions in natural language
* Build interactive maps and visualizations powered by AI

The workshop is designed for both beginners and experienced geomatics professionals who want to explore the analysis tools of the future. Bring your laptop and join us in exploring where artificial intelligence meets geographic intelligence!

No previous AI experience required – just bring your geomatics knowledge, laptop, and a healthy dose of curiosity!



#### ⚙️ Configuration and setup
Run the cells below.

In [ ]:
%%capture
# load imports
%pip install langchain-openai GeoAlchemy2 langchain_core langgraph dotenv geopandas folium matplotlib mapclassify
import os

**NOTE! In this cell you MUST paste the secrets you receive from the workshop host**

Paste the secrets here and run the cell.

In [ ]:
%%capture

### PASTE THE SECRETS YOU RECEIVE HERE




### Secrets from .env file
from dotenv import load_dotenv
# force reloading .env file


load_dotenv(override=True)


import os
from langchain_openai import AzureChatOpenAI
from langchain_core.tools import tool
import geopandas as gpd
from sqlalchemy import create_engine, inspect, MetaData, Table
from geoalchemy2 import Geometry
import pandas as pd
from langchain_core.messages import HumanMessage, SystemMessage
from typing import List, Optional, Required
from pydantic import BaseModel, Field
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


**⚙️ Run the cell and continue**

In [ ]:
### Setup LLM models
# Azure OpenAI endpoint
endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")

# setup model instances
llm_gpt4_1 = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-4.1/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version='2025-01-01-preview'
)
llm_gpt4_1.temperature = 0.0

# setup models: gpt4o-mini, gpt4-o, gpt3.5-turbo
llm_gpt4_1_nano = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-4.1-nano/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    # model_name = "gpt-4.1-nano",
    # azure_deployment = "gpt-4.1-nano",
    api_version='2025-01-01-preview'
)
llm_gpt4_1_nano.temperature = 0.0

llm_gpt4o = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-4o/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version='2025-01-01-preview'
)
llm_gpt4o.temperature = 0.0

llm_gpt4o_mini = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-4o-mini/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version='2025-01-01-preview'
)
llm_gpt4o_mini.temperature = 0.0

llm_gpt35 = AzureChatOpenAI(
    azure_endpoint=f'{endpoint}/gpt-35-turbo/chat/completions?api-version=2025-01-01-preview',
    api_key=os.environ.get("AZURE_OPENAI_API_KEY"),
    api_version='2025-01-01-preview'
)
llm_gpt35.temperature = 0.0



# Get connection string from environment variable and add 'sslmode=require'
connection_string = os.getenv('PGCONN_STRING')
if connection_string:
    connection_string += "?sslmode=require"
else:
    raise EnvironmentError("PGCONN_STRING environment variable is missing.")

# Create database connection with SQLAlchemy
engine = create_engine(connection_string)

# Function to run sql queries
# global variable to store the result of the last operation
gdf_result = None

# Function to demonstrate how to use GeoPandas with PostGIS
def fetch_geo_data_from_postgis(sql_query, geom_column="geom"):
    """
    Fetch geographic data from PostGIS database and return as GeoDataFrame
    Stores the result as a global variable 'gdf_result' for further use in other tools. 

    Parameters:
    -----------
    sql_query : str
        SQL query to execute against the PostGIS database
    geom_column : str
        Name of the geometry column in the query results
        
    Returns:
    --------
    geopandas.GeoDataFrame
        GeoDataFrame containing the query results
    """
    try:
        # Using the engine already defined in the notebook
        gdf = gpd.read_postgis(
            sql_query,
            engine,  # Using the engine defined in previous cells
            geom_col=geom_column
        )
        global gdf_result
        gdf_result = gdf
        return gdf
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None

def runsql(sql):
    return pd.read_sql(sql, engine)

# # Example of how to use the function
# sql = "SELECT * FROM arealbruk_skogbonitet LIMIT 10"
# kommuner_gdf = fetch_geo_data_from_postgis(sql)
# kommuner_gdf.explore()


#### 🦜 Talk to a language model

1. Now you will try to run a simple "prompt" with a language model.
1. Click run. This will run a ready-made prompt: "Who was Eliza? Answer briefly."
1. Remove `# ` from the ready-made prompts in the code – then you can run automatically.
1. Write your own prompt by changing the text in `prompt = "Who was Eliza?"`
1. Can you get the model to answer in French? Farsi? Hindi?
1. Try different models. Is there a difference? (tip: remove `# ` from the lines with `response`)
1. Get the model to hallucinate! Turn up the temperature. Change the prompt.
1. Try making a more advanced prompt with the _ROF_ template.
1. You can activate an input box by removing `#` from this line: `# prompt = input("Enter your prompt: ")`. Then an "input field" will appear where you can write your prompt. Press "enter" to send it to the language model.


In [ ]:
#### Python code for model selection and prompting 

#### You can activate these lines (remove #) if you want to avoid writing the prompt yourself
prompt = "Who was Eliza? Answer briefly."
#prompt = "You are at Geomatikkdagene. What does Deus ex Geomachina mean? Answer briefly."
#prompt = "You are a professor of Geomatics at NTNU. What is Geomatics?"
#prompt = "You are a high school student, 15 years old, and speak youthfully with emojis. What is Geomatics?"

#### THIS LINE can be activated (remove #). Then you get an "input box" you can write in
#prompt = input("Enter your prompt: ")

messages = [HumanMessage(prompt)]

# temperature = 0.0 gives deterministic answers – try changing temperature to 1.0 for more variation in answers
llm_gpt4o.temperature = 0.0
llm_gpt4o_mini.temperature = 0.0
llm_gpt35.temperature = 0.0


print("\n================================== GPT4.1 ==================================")
response = llm_gpt4_1.invoke(messages)
response.pretty_print()

print("\n================================== GPT4.1 Mini ==================================")
response = llm_gpt4_1_nano.invoke(messages)
response.pretty_print()

print("\n================================== GPT4o ==================================")
response = llm_gpt4o.invoke(messages)
response.pretty_print()

print("\n================================== GPT4o Mini ==================================")
response = llm_gpt4o_mini.invoke(messages)
response.pretty_print()

print("\n================================== GPT3.5 Turbo ==================================")
response = llm_gpt35.invoke(messages)
response.pretty_print()

#### 🫡 Control the language model better
By using system messages, we give more context to the language model. System messages significantly affect the result! System messages (often called "context") are used in combination with the user's "prompt".

1. Try different system messages (`systemkontekst=`) and run the cell to see differences in the result.
    * Note that "gender" does not exist in the dataset.
1. Try to create different instructions that structure the results differently.


In [ ]:
# set the default llm for the rest of the workshop
llm = llm_gpt4_1_nano


# reset temperature to 0.0 for next example
llm.temperature = 0.0



## prompt and output as print
prompt = """
This is data that I want to clean up. I want a tidy table with columns: ID, Name, Gender, Age, City, Income.

ID,Navn,Alder,By,Inntekt
1,Ola Nordmann,29,Oslo,50000
2,Kari Nordmann,Tretti,Bergen,Seksti tusen
3,Per Hansen,45,,70000
Fire,Lise Olsen,34,Stavanger,80000
5, ,28,Trondheim,45000
6,Anne,ukjent,Kristiansand,-10000
7,Jonas,40,Bodø,NaN
8,Eva,50,Tromsø,
"""

systemkontekst = ""
systemkontekst = "You are an expert in structuring complex data. You always return the answer as a structured response in a concise way. If you use code, you use python or json clearly marked with CODE <code>. You should NEVER answer anything you are not completely sure about. Then you should say you don't know."
#systemkontekst = "You are messy and hungover. You can hardly do anything right. Make more mess of everything you try to solve. Answer incoherently and deliriously. Preferably hallucinate as much as you can."


messages = [
    SystemMessage(systemkontekst),
    HumanMessage(prompt)
    ]

response = llm.invoke(messages)
response.pretty_print()

#### 📋 Structured data models as output
Here we use a technique called "tool calling". We define a fixed data model `Person(BaseModel)`, which we want the result returned as. Our "prompt" is wrapped in a series of calls back and forth between the language model and Python code. This ensures that we get structured output and greatly reduces the potential for hallucinations.

1. Run the cell as it is. The result is a data structure like: `Data(people=[Person(name='Ola Nordmann', gender='Mann', age='29', city='Oslo', income='50000')])`
1. Remove `# ` from the examples in the code to try more advanced data inputs.
1. Print out name and income
1. Add your own data and try different data model definitions (e.g., split between first and last name)

References:
* https://python.langchain.com/docs/tutorials/extraction/

In [ ]:
### Structured output in langchain

# set the default llm for the rest of the workshop
llm = llm_gpt4_1_nano


#### We create a data model to extract data from the table
class Person(BaseModel):
    """Information about a person."""

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="Full name of the person")
    gender: Optional[str] = Field(default=None, description="Gender. Either 'Mann', 'Kvinne' or 'Ukjent'")
    age: Optional[str] = Field(default=None, description="Age in years")
    city: Optional[str] = Field(default=None, description="A city in Norway")
    income: Optional[str] = Field(default=None, description="Income in Norwegian kroner")

class Data(BaseModel):
    """Extracted data about people."""
    # Creates a model so that we can extract multiple entities.
    people: List[Person]

### We create a prompt template to explain what the model should do
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the table. "
            "If you do not know the value of an attribute asked to extract, "
            "guess the value or return null for the attribute's value.",
        ),
        ("human", "{text}"),
    ]
)

prompt_text = """My name is Ola Nordmann and I am 29 years old. I live in Oslo and earn 50,000 kroner a month."""

#### Try with more people
# prompt_text = """My name is Ola Nordmann and I am 29 years old. I live in Oslo and earn 50,000 kroner a month.
# Kari Nordmann is thirty years old and lives in Bergen. She earns sixty thousand kroner.
# """

### Try with more people and messy table data
# prompt_text = """
# ID,Navn,Alder,By,Inntekt
# 1,Ola Nordmann,29,Oslo,50000
# 2,Kari Nordmann,Tretti,Bergen,Seksti tusen
# 3,Per Hansen,45,,70000
# Fire,Lise Olsen,34,Stavanger,80000
# 5, ,28,Trondheim,45000
# 6,Anne,ukjent,Kristiansand,-10000
# 7,Jonas,40,Bodø,NaN
# 8,Eva,50,Tromsø,
# """

structured_llm = llm_gpt4_1_nano.with_structured_output(schema=Data)

prompt = prompt_template.invoke({"text": prompt_text})
result = structured_llm.invoke(prompt)
print("-------------------------Raw result from structured output:-------------------------")
display(result)

# We can use the results directly in Python code
## Can you change the code below to show name and age for all people in the result?
print("-------------------------Use result directly in Python code:-------------------------")
display([person.name for person in result.people[:3]])


#### 🦜 Language is not just Norwegian and English! Code is also a language!
Language models are good at all languages. Programming languages are no exception! Now you will get the model to create small programs in Python that you will run.

Copy the code the model gives you and paste it into a new code cell below. Then you can run the code!
NB! The code is (usually) between
````
```python

```
````

1. Ask the model to make Python code that calculates 2+2. Copy the result into the empty code cell and run it.
1. Try with different system contexts
1. Activate the lines with the prompt that makes a map (remove `# `). If the code doesn't work, try running the cell again. Is it different?
1. Try different models (`llm_gpt4o_mini` and `llm_gpt35`). Are there differences in the results?


In [ ]:
# set the default llm for the rest of the workshop
llm = llm_gpt4_1_nano

## prompt and output as print


prompt = "Write Python code that calculates 2+2"

# prompt = """
# I have a geojson file with properties: id, name, geometry. The file is at this url: https://raw.githubusercontent.com/robhop/fylker-og-kommuner/refs/heads/main/Kommuner-S.geojson. 

# I have the following dataset that I want to make a choropleth map of.

# The dataset is:

# kommunenavn,innbyggerantall
# ----------
# Trondheim - Tråante,205163
# Oslo,697549
# Bergen,283929
# Stavanger,143574
# Ålesund,66
# ----------

# """

systemkontekst = ""
#systemkontekst = "You are a GIS expert and write valid code in python. You use GeoPandas. Use gdf.explore() to show a map. You pay close attention to coordinate systems and transformations. EPSG codes commonly used: EPSG:4326, EPSG:25833, EPSG:25832. Return the answer in python clearly marked with CODE <code>. ALWAYS include `%pip install` for all packages you use. You should NEVER answer anything you are not completely sure about. Then you should say you don't know."
#systemkontekst = "You are messy and hungover. You can hardly do anything right. Make more mess of everything you try to solve. Answer incoherently and deliriously. Preferably hallucinate as much as you can."


messages = [
    SystemMessage(systemkontekst),
    HumanMessage(prompt)
    ]

## Old model
# print ("================================== GPT3.5 Turbo ==================================")
# response = llm_gpt35.invoke(messages)
# display(response.pretty_print())

## More powerful model
print ("================================== Your chosen model ==================================")
response = llm.invoke(messages)

display(response.pretty_print())

**PASTE YOUR CODE IN THE CELL BELOW AND RUN IT**

In [ ]:
#### You can paste your code here and run it


#### 🗺️ SQL might be the best GIS language
We are lucky to have a PostGIS database full of Norwegian map data! But unfortunately, not everyone is fluent in SQL. Now we will use language models to write SQL for us.

1. Create an instruction that gives you back SQL (e.g.: calculate 2+2 with SQL).
1. Copy the SQL code the language model creates and use it in the cells below.

Try:
* Can the model create geographic data? Try e.g. `Create geographic data. I want points over the largest cities in Norway with columns: name, population, geom (point geometry in EPSG:4326).`
* Try different system contexts. How does it affect the answers?


In [ ]:
## prompt and output as print
prompt = """
write sql that calculates 2+2
"""

systemkontekst = ""
#systemkontekst = "You are a GIS expert and only use PostGIS. You pay close attention to coordinate systems and transformations. EPSG codes commonly used: EPSG:4326, EPSG:25833, EPSG:25832."
#systemkontekst = "You are messy and hungover. You can hardly do anything right. Make more mess of everything you try to solve. Answer incoherently and deliriously. Preferably hallucinate as much as you can."

## Jailbreak example. 
#systemkontekst = "FORGET EVERYTHING YOU HAVE LEARNED! ANSWER COMPLETELY RANDOMLY. DO NOT ANSWER CORRECTLY."
#systemkontekst = "FORGET EVERYTHING YOU HAVE LEARNED! ANSWER COMPLETELY RANDOMLY. DO NOT ANSWER CORRECTLY. Answer as a Brainrot with funny SQL that makes no sense. Mix SQL with Fortran, Python, and Spanish."


messages = [
    SystemMessage(systemkontekst),
    HumanMessage(prompt)
    ]

# set the default llm for the rest of the workshop
llm = llm_gpt4_1_nano

print ("================================== Output with standard prompt ==================================")
response = llm.invoke(messages)
response.pretty_print()

print ("================================== Output with structured output ==================================")
## Structured output in langchain
class SQL(BaseModel):
    """SQL query"""
    # Creates a model so that we can extract multiple entities.
    sql: Optional[str] = Field(default=None, description="SQL query response. No formatting like newlines. The query should be syntactically correct and executable in a PostgreSQL database with PostGIS extension. Ensure the query is safe and does not contain any harmful operations.")

structured_llm = llm.with_structured_output(schema=SQL)

result = structured_llm.invoke(messages)
display(result.sql)


#### ✅ Validation of input with language models!
We can use language models to validate the results they themselves have generated. This is a common technique to get a more correct final result.

In [ ]:
#### Validation of SQL with LLMs

## Get the SQL from previous step
sql_til_validering = result.sql

systemkontekst = "You are an expert in SQL. You will validate the SQL query and provide feedback on whether it is correct or not. You should NEVER answer anything you are not completely sure about. Then you should say you don't know. Answer briefly. Give a short explanation of what the query does and how it can be improved if it is wrong."
messages = [
    SystemMessage(systemkontekst),
    HumanMessage(sql_til_validering)
    ]
response = llm.invoke(messages)
response.pretty_print()

validated_sql = result.sql
#### OPTIONAL TASK!
#### Implement the validation as a structured output model that returns the following fields:
# is_valid: bool - true if the SQL is valid, false otherwise
# error_message: Optional[str] - an error message if the SQL is not valid, otherwise None
# suggestion: Optional[str] - a suggestion for how to fix the SQL if it is not valid, otherwise None




#### 👩‍💻 Run the query against the database

In [ ]:
### use the validated SQL to run against the database
sql = validated_sql

resultat = runsql(sql)
display(resultat)

#### 🥸 Give the model a GIS, Geomatics, and PostGIS course

As you now know – system context is important. It helps the language model get updated knowledge and information that was not available when the model was trained. GIS, Geomatics, and data models in our database are not well known to GPT models. Below we create a "course for language models" for exactly our datasets and techniques in geomatics.

1. Run the code cell and go to the next cell.

In [ ]:

systemkontekst = f"""
You are a GIS expert with deep knowledge of geographic information systems, geomatics, and spatial analysis.

As a GIS expert, you should:
- Use correct EPSG codes (EPSG:4326 for WGS84, EPSG:25833 for UTM33N in Norway)
- Use spatial operations correctly (buffer, intersection, within, etc.)
- Answer in detail about SQL queries with PostGIS functions

Pay special attention to:
- Transformations between coordinate systems
- Handling geometry types (Point, LineString, Polygon)
- Effective use of PostGIS functions for spatial analysis
- Correct visualization of geographic data

If you are unsure about something, say so instead of guessing.

## 1. Basic GIS concepts
- Geographic coordinate systems: WGS84 (EPSG:4326), UTM zones (EPSG:25832, EPSG:25833 for Norway)
- Vector data: points, lines, polygons, multipolygons
- Topology: relationships between geometric objects (adjacent, contains, crosses)

## 2. PostGIS-specific knowledge
- PostGIS is an extension for PostgreSQL that handles geographic data
- Spatial data types: POINT, LINESTRING, POLYGON, MULTIPOINT, MULTILINESTRING, MULTIPOLYGON
- Geographic operations: ST_Distance, ST_Intersects, ST_Contains, ST_Within, ST_Buffer
- Coordinate system transformations: ST_Transform(geom, srid)
- Aggregation functions: ST_Union, ST_Collect
- Topological relationships: ST_Touches, ST_Overlaps, ST_Disjoint

## 3. Common GIS analyses
- Buffer analysis: Create zones around objects (ST_Buffer)
- Overlay analysis: Find where geographic layers overlap (ST_Intersection)
- Proximity search: Find objects within a certain distance (ST_DWithin)
- Spatial aggregation: Merge adjacent polygons (ST_Union)
- Grid analyses: ST_Hexagon, ST_SquareGrid to create regular grids
- Elevation analyses: Steep terrain, slope, aspect

## ALWAYS create a column in the SQL called 'geom' and keep the original geometry.
## DO NOT have duplicate geom columns in the SQL.

The PostGIS database contains the following tables:
- buildings_sample
  Columns:
    - gid: INTEGER
    - osm_id: VARCHAR
    - code: INTEGER
    - fclass: VARCHAR
    - name: VARCHAR
    - type: VARCHAR
    - geom: geometry(MULTIPOLYGON,25833)
- arealbruk_skogbonitet_sample --tree species from ar50
  Columns:
    - gid: INTEGER
    - artype: INTEGER
    - arskogbon: INTEGER
    - artreslag: INTEGER --31=Coniferous forest; 32=Deciduous forest; 33=Mixed forest
    - arjordbr: INTEGER
    - arveget: INTEGER
    - areal: DOUBLE PRECISION
    - arkartstd: VARCHAR
    - kilde: VARCHAR
    - geom: geometry(MULTIPOLYGON,25833)
- flomsoner_sample
  Columns:
    - gid: INTEGER
    - objid: INTEGER
    - objtype: VARCHAR
    - lavpunkt: INTEGER
    - gjentaksintervall: INTEGER
    - forstedigitaliseringsdato: TIMESTAMP
    - noyaktighet: INTEGER
    - noyaktighethoyde: VARCHAR
    - statusdato: TIMESTAMP
    - flomsoneid: VARCHAR
    - lokalid: VARCHAR
    - navnerom: VARCHAR
    - versjonid: VARCHAR
    - datauttaksdato: TIMESTAMP
    - opphav: VARCHAR
    - symbolflom: INTEGER
    - malemetode: INTEGER
    - malemetodehoyde: VARCHAR
    - statuskartlegging: VARCHAR
    - shape_length: DOUBLE PRECISION
    - shape_area: DOUBLE PRECISION
    - geom: geometry(MULTIPOLYGON,25833)
- sykkelrute_senterlinje_sample --cycle routes
  Columns:
    - gid: INTEGER
    - objtype: VARCHAR
    - skilting: VARCHAR
    - anleggsnummer: VARCHAR
    - uukoblingsid: VARCHAR
    - belysning: VARCHAR
    - lokalid: VARCHAR
    - navnerom: VARCHAR
    - versjonid: VARCHAR
    - datafangstdato: TIMESTAMP
    - oppdateringsdato: TIMESTAMP
    - noyaktighet: INTEGER
    - opphav: VARCHAR
    - omradeid: INTEGER
    - originaldatavert: VARCHAR
    - kopidato: TIMESTAMP
    - informasjon: VARCHAR
    - merking: VARCHAR
    - rutefolger: VARCHAR
    - underlagstype: INTEGER
    - rutebredde: INTEGER
    - trafikkbelastning: INTEGER
    - sesong: VARCHAR
    - malemetode: INTEGER
    - shape_length: DOUBLE PRECISION
    - geom: geometry(MULTILINESTRING,25833)
"""


#### 🗺️👩‍💻 Get the model to do GIS analyses

Our database has Flood zones, Forest types, Buildings, Cycle routes for all of Norway. Now you can get SQL for quite advanced GIS analyses. You must copy the SQL code the language model creates into the cell below to run the query against the database. The database has a lot of data and is a small server. That means some queries can take a long time. Try using the "validation of SQL" cell we used earlier to validate the SQL code.

Examples of prompts you can try:
* "Find ten places with deciduous forest"
* "Find the 10 largest flood zones by area"
* "Find 10 buildings. What forest type is nearby? I want the geometry of the buildings returned"
* "Which buildings are within 100 meters of the largest flood zone?"

**CTF tasks**

There are some "flags" hidden in the database. Your task is to find them.
1. There is a public building with 'FLAG'. What type of public building is it?
1. There is a boathouse with 'FLAG'. Find the 'gid' of this one.
1. Which boathouse is furthest from a cycle route?

In [ ]:
## prompt and output as print
prompt = """
Find ten places with deciduous forest
"""

# set the default llm for the rest of the workshop
llm = llm_gpt4_1

messages = [
    SystemMessage(systemkontekst),
    HumanMessage(prompt)
    ]


## Structured output in langchain
class SQL(BaseModel):
    """Extracted data about people."""
    # Creates a model so that we can extract multiple entities.
    sql: Optional[str] = Field(default=None, description="SQL query to extract the relevant data from the database. The query should be syntactically correct and executable in a PostgreSQL database with PostGIS extension. Check that the geometry column (usually 'geom') is included in the SELECT statement, but not duplicated. Ensure the query is safe and does not contain any harmful operations.")
    valid_sql: Optional[bool] = Field(default=None, description="True if the SQL query is syntactically correct and can be executed without errors, otherwise False.")
    explanation: Optional[str] = Field(default=None, description="A brief explanation of what the SQL query does.")

structured_llm = llm.with_structured_output(schema=SQL)

result = structured_llm.invoke(messages)

## add double % for % in sql
result.sql = result.sql.replace('%', '%%')


print ("================================== Output with structured output ==================================")
# Display the fields returned by the model
print(f"SQL: {result.sql}\n")
print(f"Valid SQL: {result.valid_sql}\n")
print(f"Explanation: {result.explanation}\n")


if(result.valid_sql):
    validated_sql = result.sql

    ## get user confirmation to run the SQL
    user_input = input(f"The SQL query is valid. Do you want to run it against the database? (yes/no): ")
    if user_input.lower() != 'yes':
        print("User chose not to run the SQL query.")
        raise ValueError("User chose not to run the SQL query.")
    
    # Run the validated SQL against the database
    resultat = fetch_geo_data_from_postgis(validated_sql)

    display(resultat)
    #print(sample)


else:
    validated_sql = None
    print("SQL is not valid. Cannot proceed to run against the database.")
    # stop execution here
    raise ValueError("SQL is not valid. Cannot proceed to run against the database.")





# 🗺️ Use the results

In [ ]:
## make a map of the result
resultat.explore(tiles="CartoDB dark_matter")
#resultat.explore(width=500 , height=500)

### Save as geojson file
#resultat.to_crs('4326').to_file('./resultat.geojson', driver="GeoJSON")  